In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['101_objectcategories']


In [25]:
print(os.listdir("../input/101_objectcategories/101_ObjectCategories/"))
print(os.listdir("/tmp"))

['dollar_bill', 'buddha', 'platypus', 'euphonium', 'okapi', 'barrel', 'water_lilly', 'mandolin', 'cup', 'trilobite', 'scissors', 'elephant', 'tick', 'car_side', 'pigeon', 'watch', 'hedgehog', 'strawberry', 'stegosaurus', 'pizza', 'gerenuk', 'crocodile_head', 'flamingo_head', 'inline_skate', 'cannon', 'anchor', 'rooster', 'bonsai', 'mayfly', 'wild_cat', 'umbrella', 'crab', 'ant', 'grand_piano', 'ibis', 'menorah', 'ceiling_fan', 'accordion', 'brontosaurus', 'dolphin', 'panda', 'ferry', 'sunflower', 'ewer', 'garfield', 'hawksbill', 'BACKGROUND_Google', 'lamp', 'yin_yang', 'emu', 'cougar_body', 'headphone', 'airplanes', 'dalmatian', 'rhino', 'soccer_ball', 'lobster', 'pagoda', 'joshua_tree', 'windsor_chair', 'nautilus', 'camera', 'Faces_easy', 'saxophone', 'octopus', 'wheelchair', 'bass', 'schooner', 'lotus', 'cellphone', 'beaver', 'Motorbikes', 'wrench', 'electric_guitar', 'gramophone', 'chandelier', 'pyramid', 'minaret', 'crocodile', 'chair', 'sea_horse', 'Leopards', 'metronome', 'staple

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')


In [27]:
train_generator=train_datagen.flow_from_directory("../input/101_objectcategories/101_ObjectCategories/",batch_size=20,target_size=(256,256),
                                                  class_mode='categorical')


Found 9144 images belonging to 102 classes.


In [28]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (256, 256, 3), 
                                include_top = False, 
                                weights = 'imagenet')

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

#last_layer = pre_trained_model.get_layer('mixed7')
#print('last layer output shape: ', last_layer.output_shape)
#last_output = last_layer.output

--2019-06-10 02:31:31--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 2607:f8b0:400c:c12::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   144MB/s    in 0.6s    

2019-06-10 02:31:32 (144 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [29]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


last layer output shape:  (None, 14, 14, 768)


In [31]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (101, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = 'adam', 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])


In [32]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 127, 127, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1_94 (Batc (None, 127, 127, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 127, 127, 32) 0           batch_normalization_v1_94[0][0]  
__________________________________________________________________________________________________
conv2d_95 

In [10]:
history=model.fit_generator(train_generator,steps_per_epoch=100,epochs=1)

Epoch 1/5
 84/458 [====>.........................] - ETA: 2:11 - loss: 0.0558 - acc: 0.9881

KeyboardInterrupt: 